In [25]:
import pandas as pd
import os

In [26]:
!gsutil -m cp -R gs://urm-pr-experiment.appspot.com/experiment_results/* ./data

Copying gs://urm-pr-experiment.appspot.com/experiment_results/silence_1_stroop_17aa613e-b4c9-49de-8429-f1033be748fd_1672933193552.csv...
Copying gs://urm-pr-experiment.appspot.com/experiment_results/silence_1_stroop_17aa613e-b4c9-49de-8429-f1033be748fd_1672934280337.csv...
Copying gs://urm-pr-experiment.appspot.com/experiment_results/silence_1_stroop_17aa613e-b4c9-49de-8429-f1033be748fd_1672935782742.csv...
Copying gs://urm-pr-experiment.appspot.com/experiment_results/silence_2_digit_span_17aa613e-b4c9-49de-8429-f1033be748fd_1672935902676.csv...
Copying gs://urm-pr-experiment.appspot.com/experiment_results/silence_3_symmetry_span_17aa613e-b4c9-49de-8429-f1033be748fd_1672936202964.csv...
/ [5/5 files][184.5 KiB/184.5 KiB] 100% Done                                    
Operation completed over 5 objects/184.5 KiB.                                    


In [27]:
data_dir = 'data'
file_names = [f for f in os.listdir(data_dir) if f.endswith('.csv') and (f.startswith('music') or f.startswith('silence'))]
file_paths = [os.path.join(data_dir, file) for file in file_names]
file_paths

['data/music_1_digit_span_4d1a0714-8767-4a2f-9fc4-10c11b7c31dd_1672839998434.csv',
 'data/silence_1_stroop_17aa613e-b4c9-49de-8429-f1033be748fd_1672933193552.csv',
 'data/silence_2_digit_span_17aa613e-b4c9-49de-8429-f1033be748fd_1672935902676.csv',
 'data/silence_1_stroop_17aa613e-b4c9-49de-8429-f1033be748fd_1672935782742.csv',
 'data/music_2_symmetry_span_4d1a0714-8767-4a2f-9fc4-10c11b7c31dd_1672840020504.csv',
 'data/silence_1_stroop_17aa613e-b4c9-49de-8429-f1033be748fd_1672934280337.csv',
 'data/silence_3_symmetry_span_17aa613e-b4c9-49de-8429-f1033be748fd_1672936202964.csv',
 'data/silence_2_symmetry_span_8d3e33f4-7baf-43ca-b627-a1d69bcda56a_1672840360373.csv',
 'data/silence_1_digit_span_8d3e33f4-7baf-43ca-b627-a1d69bcda56a_1672840340535.csv']

In [ ]:
for file_path in file_paths:
    df = pd.read_csv(file_path)

In [13]:
pd.read_csv(file_paths[0])

,rt,recall,stimuli,accuracy,trial_type,trial_index,time_elapsed,internal_node_id
0,3062,"1,9,5","1,9,5",1,digit-span-recall,5,9884,0.0-2.0-1.0
1,2956,"7,1,2,9","7,1,2,9",1,digit-span-recall,11,17883,0.0-2.0-1.1


In [10]:
data = pd.read_csv(file_paths[1])

In [18]:
data.success.isna().sum()

228

In [14]:
trials = data.query('type == "test trial"')
trials

,success,trial_type,trial_index,time_elapsed,internal_node_id,view_history,rt,stimulus,key_press,type,Congruency,letter,correct
101,NaN,html-keyboard-response,101,110482,0.0-7.0-1.0,NaN,989.7,"<table border=""0"" width=""600""><tr height=""300""...",86.0,test trial,Incon,v,True
103,NaN,html-keyboard-response,103,112826,0.0-7.0-1.1,NaN,1086.0,"<table border=""0"" width=""600""><tr height=""300""...",78.0,test trial,Incon,n,True
105,NaN,html-keyboard-response,105,114467,0.0-7.0-1.2,NaN,631.3,"<table border=""0"" width=""600""><tr height=""300""...",66.0,test trial,Incon,b,True
107,NaN,html-keyboard-response,107,116410,0.0-7.0-1.3,NaN,934.2,"<table border=""0"" width=""600""><tr height=""300""...",86.0,test trial,Incon,v,True
109,NaN,html-keyboard-response,109,117506,0.0-7.0-1.4,NaN,838.5,"<table border=""0"" width=""600""><tr height=""300""...",77.0,test trial,Incon,m,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,NaN,html-keyboard-response,219,222436,0.0-7.0-1.59,NaN,584.1,"<table border=""0"" width=""600""><tr height=""300""...",78.0,test trial,Incon,n,True
221,NaN,html-keyboard-response,221,224341,0.0-7.0-1.60,NaN,401.5,"<table border=""0"" width=""600""><tr height=""300""...",78.0,test trial,Incon,n,True
223,NaN,html-keyboard-response,223,225654,0.0-7.0-1.61,NaN,802.1,"<table border=""0"" width=""600""><tr height=""300""...",77.0,test trial,Incon,m,True
225,NaN,html-keyboard-response,225,227676,0.0-7.0-1.62,NaN,512.3,"<table border=""0"" width=""600""><tr height=""300""...",78.0,test trial,Incon,n,True


## Making sense of stimulus and letter columns

In [34]:
def extract_rgb(stimulus):
    return [int(color_str) for color_str in stimulus.split("rgb(")[1].split(")")[0].split(",")]

def get_color(rgb_vals):
    if (rgb_vals[0] == 255 and rgb_vals[1] == 0 and rgb_vals[2] == 0):
        return 'red'
    elif (rgb_vals[0] == 255 and rgb_vals[1] == 255 and rgb_vals[2] == 0):
        return 'yellow'
    elif (rgb_vals[0] == 0 and rgb_vals[1] == 255 and rgb_vals[2] == 0):
        return 'green'
    elif (rgb_vals[0] == 0 and rgb_vals[1] == 0 and rgb_vals[2] == 255):
        return 'blue'
    else:
        return 'unknown'

def key_to_color(key):
    if key == 'v':
        return 'red'
    elif key == 'b':
        return 'yellow'
    elif key == 'n':
        return 'green'
    elif key == 'm':
        return 'blue'
    else:
        return 'unknown'

def extract_true_and_detected(row):
    rgb_vals = extract_rgb(row.stimulus)
    true_color = get_color(rgb_vals)
    key = row.letter
    detected_color = key_to_color(key)
    return pd.Series([true_color, detected_color])


results = trials.apply(extract_true_and_detected, axis=1, result_type='expand').rename(columns={0: 'true_color', 1: 'detected_color'})
results

,true_color,detected_color
101,red,red
103,green,green
105,yellow,yellow
107,red,red
109,blue,blue
...,...,...
219,green,green
221,green,green
223,blue,blue
225,green,green


In [35]:
results.true_color.value_counts()

red       16
green     16
yellow    16
blue      16
Name: true_color, dtype: int64

In [36]:
results.detected_color.value_counts()

red       16
green     16
yellow    16
blue      16
Name: detected_color, dtype: int64

In [28]:
trials.correct

101     True
103     True
105     True
107     True
109     True
       ...  
219     True
221     True
223     True
225     True
227    False
Name: correct, Length: 64, dtype: object